In [1]:
!pip install --upgrade transformers
!pip install pytorchvideo evaluate -q
!pip install --upgrade datasets
!pip install decord
!pip install -U huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.4 MB/s eta 0

In [2]:
from prediction import *

In [10]:
# load the model from disk
import pickle
filename = "/content/finalized_model.sav"
clf = pickle.load(open(filename, 'rb'))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [16]:
target_names = {0:'ang',
                1:'hap',
                2:'neu',
                3:'sad'}

In [17]:
text_mdl = "minoosh/finetuned_bert-base-on-IEMOCAP_2"
speech_mdl = "minoosh/finetuned_wav2vec2.0-base-on-IEMOCAP_2"
video_mdl = "minoosh/videomae-base-finetuned-IEMOCAP_2"

In [18]:
load_models(text_mdl, speech_mdl, video_mdl)

Some weights of the model checkpoint at minoosh/finetuned_bert-base-on-IEMOCAP_2 were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at minoosh/finetuned_wav2vec2.0-base-on-IEMOCAP_2 were not used when initializing Wav2Vec2Model: ['projector.bias', 'classifier.bias', 'classifier.weight', 'projector.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertFor

In [6]:
from huggingface_hub import hf_hub_download
import pathlib

hf_dataset_identifier = "minoosh/MELD_test"
filename = "MELD_test"
file_path = hf_hub_download(
    repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset"
)

!tar xf {file_path}

In [34]:
import pandas as pd
meld_test = pd.read_csv('meld_test.csv')

In [35]:
list_of_perdictions = []
list_of_y = []

In [36]:
meld_test

,Emotion,name
0,0,dia0_utt1
1,2,dia0_utt2
2,2,dia1_utt0
3,1,dia1_utt1
4,1,dia1_utt2
...,...,...
2206,0,dia279_utt10
2207,2,dia279_utt11
2208,2,dia279_utt13
2209,2,dia279_utt14


In [37]:
#delete rows with number of frames <= 16

for row in meld_test.iterrows():
  name = row[1]['name']
  video_file_path = '/content/video/' + name + '.mp4'
  container = av.open(video_file_path)
  fps = container.streams.video[0].frames
  f.append(fps)
  if fps<=16:
    meld_test = meld_test.drop(row[0])
  elif fps >= 7000:
    meld_test = meld_test.drop(row[0])

In [38]:
meld_test = meld_test.reset_index(drop=True)

In [39]:
meld_test

,Emotion,name
0,0,dia0_utt1
1,2,dia0_utt2
2,2,dia1_utt0
3,1,dia1_utt1
4,1,dia1_utt2
...,...,...
2098,0,dia279_utt10
2099,2,dia279_utt11
2100,2,dia279_utt13
2101,2,dia279_utt14


In [40]:
for row in meld_test.iterrows():

  #print(row[0])
  name = row[1]['name']
  text_file_path = '/content/text/' + name+'.txt'
  speech_file_path = '/content/speech/' + name + '.wav'
  video_file_path = '/content/video/' + name + '.mp4'
  torch.cuda.empty_cache()
  x = prepare_data(text_file_path, speech_file_path, video_file_path)
  y = clf.predict(x.cpu())
  list_of_perdictions.append(y[0])
  list_of_y.append(row[1]['Emotion'])

Streaming output truncated to the last 5000 lines.
/content/prediction.py:79: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = {k:torch.tensor(v).to(device) for k,v in inputs.items()}
/content/prediction.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text_inputs = {k:torch.tensor(v).to(device) for k,v in text_inputs.items()}
/content/prediction.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  speech_input = {k:torch.tensor(v).to(device) for k,v in speech_input.items()}
/content/prediction.py:79: UserWarning: To copy constr

In [41]:
len(list_of_perdictions)

2103

In [42]:
meld_test['predicted'] = list_of_perdictions

In [43]:
meld_test.to_csv('meld_test_and_predictions.csv', index=False)

In [44]:
from sklearn.metrics import classification_report

target_names = ['ang', 'hap', 'neu', 'sad']
print(classification_report(list_of_y, list_of_perdictions, target_names=target_names, digits=6))

              precision    recall  f1-score   support

         ang   0.000000  0.000000  0.000000       325
         hap   0.250000  0.002646  0.005236       378
         neu   0.568940  0.982456  0.720588      1197
         sad   0.031250  0.004926  0.008511       203

    accuracy                       0.560152      2103
   macro avg   0.212548  0.247507  0.183584      2103
weighted avg   0.371786  0.560152  0.411912      2103



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#print(target_names[y[0]])

neu
